<a href="https://colab.research.google.com/github/CPukszta/BI-BE-CS-183-2023/blob/main/HW8/Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bi/Be/Cs 183 2022-2023: Intro to Computational Biology
TAs: Meichen Fang, Tara Chari, Zitong (Jerry) Wang

**Submit your notebooks by sharing a clickable link with Viewer access. Link must be accessible from submitted assignment document.**

Make sure Runtime $\rightarrow$ Restart and run all works without error

# **HW 8 Problem 3**

Chimpanzees are the closest living relatives to humans. Here we will be looking at differences between the two genomes. 

In this problem you will implement the Needleman-Wunsch algorithm for global alignment between a pair of sequences of human and chimpanzee FOXP2 genes [1]. You will be reading in a FASTA file of DNA sequences and run the algorithm to find their optimal alignment given a set of rewards and penalties for the possible nucleotide pairs. This algorithm allows for gaps in the alignment, and constructs the global optimal alignment by iteratively finding the solutions for sets of small subsequences and their alignments.

FOXP2 is a gene that has been implicated in speech and language development in humans, where mutations in the gene can inhibit speech. Interestingly, FOXP2s sequence and function are different between chimpanzees and humans, and its mechanism(s) is an area of study for researchers in understanding how we process and develop language.

[1] Enard W, Przeworski M, Fisher SE, Lai CS, Wiebe V, Kitano T, Monaco AP, Pääbo S. Molecular evolution of FOXP2, a gene involved in speech and language. Nature. 2002 Aug 22;418(6900):869-72. doi: 10.1038/nature01025. Epub 2002 Aug 14. PMID: 12192408.



##**Import data and install packages**

In [1]:
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt #Can use other plotting packages like seaborn

In [2]:
#Download the FASTA file with a pair of nucleotide sequences

!wget --content-disposition 'https://drive.google.com/uc?export=download&id=1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm'

--2023-03-07 03:54:34--  https://drive.google.com/uc?export=download&id=1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm
Resolving drive.google.com (drive.google.com)... 173.194.217.138, 173.194.217.100, 173.194.217.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.217.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mv7rue7pahq8qd0scmm8ejt6hthk65gn/1678161225000/14515314329444727017/*/1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm?e=download&uuid=5aaf9c26-0a16-44d8-abc7-111385eb3770 [following]
--2023-03-07 03:54:34--  https://doc-0k-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mv7rue7pahq8qd0scmm8ejt6hthk65gn/1678161225000/14515314329444727017/*/1GFYciWTl2Objrplb1XqNa0pWzTBvXbsm?e=download&uuid=5aaf9c26-0a16-44d8-abc7-111385eb3770
Resolving doc-0k-3c-docs.googleusercontent.com (doc-0k-3c-docs.googleusercontent.com)... 142.250.97.132, 2

## **Read in data for analysis**

**The data**

Here you will read in  a FASTA file of a pair of DNA sequences. FASTA is a common text file format in which nucleotide sequences are stored. Commonly, the name of the sequence follows a '>'. The following line then has the sequence itself, terminated by an '*'.

In [3]:
#Print the FASTA file we downloaded
!cat seqs.fasta

>sequence1
ACAACAACAG*
>sequence2
ACAACAGCAGCAG*


## **Problem 3 (40 points)**

In the Needleman-Wunsch (NW) algorithm we will have two matrices: the score matrix and traceback matrix. 

###Score Matrix

We begin by constructing a **score matrix** $\mathbf{F}$, an $(m+1)\times(n+1)$ matrix for two sequences $x,y$ of length $m$ and $n$ respectively. Each entry $i,j$ corresponds to the alignment score of the nucleotides in at $x_i \text{ and } y_j$.

Nucleotide pairs can either be aligned to eachother (e.g. $F(1,1)$ = 'G' mapped to 'G') or to a gap (e.g. $F(1,0)$ = 'G' mapped to '-'). Pairs can also be aligned if they do not match (e.g. $F(1,2)$ = 'G' mapped to 'A'). (See below for sequences $x$ = 'GATTACA', $y$ = 'GCATGCT')







<center><img src="https://drive.google.com/uc?export=view&id=1Qfzs96bRkU2YJwx8Ss9VcjRN2EPUyLY4" alt="exGrid" width="300" height="300"><center>

To calculate a score/entry for $F_{i,j}$, we will use the scores of $F_{i-1,j-1},F_{i-1,j}$ and $F_{i,j-1}$. The score for $F_{i,j}$ is defined as the maximum score of (1) $x_i$ being aligned to $y_j$, (2) $x_i$ being aligned to a gap '-', and (3) $y_j$ being aligned to a gap '-'. Thus the score is calculated as:

\begin{align}
F_{i,j} = max\begin{cases} 
      F_{i-1,j-1} + s(x_i,y_j) & (1) \\
      F_{i-1,j} - d & (2) \\
      F_{i,j-1} - d & (3)
   \end{cases}
\end{align}

$s()$ denotes a score function, which we will define below, that returns a reward for two matched nucleotides or a penalty for mismatched bases.
We will also set a value for the 'gap penalty' $d$.

We can draw this score calculation as:

<center><img src="https://drive.google.com/uc?export=view&id=1MZf_hZ_7Tprj0hV0rpwsBWXBmrmFvWt_" alt="exScore" width="300" height="160"><center>

###Traceback Matrix

After we find out the highest score, the previous cell which gave the highest candidate score must also be recorded. This information is recorded in the **traceback matrix** $\mathbf{P}$. For example, if $F_{i-1,j-1} + s(x_i,y_j)$ gives the maximum score for $F_{i,j}$, then the corresponding cell of the traceback matrix $\mathbf{P}_{ij}$ is ”diag”, which means the last cell is $F_{i-1,j-1}$. If it is the above cell $F_{i-1,j}$ that gives the maximum score , then the $\mathbf{P}_{ij}$ is ”up”. If there are even scores, we can just record one of them for the problem.

After filling all cells of the score matrix and traceback matrix, the score in the cell on the bottom right represents the alignment score for the best alignment. We then use the traceback matrix to construct the best alignment.

The traceback begins with the bottom right cell. One moves according to the traceback value written in the cell. There are three possible moves: diagonally (toward the top-left corner of the matrix), up, or left.
The traceback is completed when the first, top-left cell of
the matrix is reached.

Here is the bottom right corner of an example traceback matrix:
<center><img src="https://drive.google.com/uc?export=view&id=183LAe4zPt2s8qsyfVmyScoWxyUBVC6uX" alt="exScore" width="250" height="160"><center>

In summary, the Needleman-Wunsch algorithm consists of three steps:
1. Initialisation of the score matrix
2. Calculation of scores and filling the traceback matrix
3. Deducing the alignment from the traceback matrix

We will implement each step below.

### **a) Read in the DNA sequences from the FASTA file. Extract just the sequence strings, *not including* the asterisk at the end, and print the two strings. (2 points)**

In [4]:
#read the FASTA
f=open('seqs.fasta')
lines=f.readlines()

#get the individual sequences
seq1 = lines[1]
seq1 = seq1[:-2]

seq2 = lines[3]
seq2 = seq2[:-2]


#print sequences
print("sequence 1:",seq1)
print("sequence 2:",seq2)


sequence 1: ACAACAACAG
sequence 2: ACAACAGCAGCAG


### **b) Initialisation of the score matrix: fill in the boundary conditions of $\mathbf{F}$. (5 points)**
The first column and row denote all bases in that column or row being mapped to gaps '-'. Assuming $F_{0,0} = 0$, fill in row 0 and column 0 accordingly, using a **gap penalty $\mathbf{d = -1}$**. 

**Print your $\mathbf{F}$ matrix after initialization.**

In [5]:
f = np.zeros((len(seq1)+1,len(seq2)+1))

for i in range((np.shape(f)[0])-1):
  f[i+1,0] = f[i,0] -1

for i in range((np.shape(f)[1])-1):
  f[0,i+1] = f[0,i] -1

print(f)

[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13.]
 [ -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -3.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -4.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -5.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -6.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -7.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -8.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [ -9.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [-10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


### **c) Implement the s() (score) function. (3 points)**

Fill in the s() function below to take in two nucleotide bases and output a **reward of 2** if the bases match, and a **penalty of -2** for mismatched bases.

In [6]:
def s(b1,b2):
  if b1==b2:
    s = 2
  else:
    s = -2
  return s

### **d) Calculation of scores and filling the traceback matrix: fill in the rest of the $\mathbf{F}$ matrix, row by row, and store the information in traceback matrix $\mathbf{P}$. (10 points)**

Fill in the $\mathbf{F}$ matrix using the score function described in the main Problem statement, using the s() function from **c)** and a **gap penalty $\mathbf{d = -1}$**
 
Remember to store the arrow or the previous cells when we fill in the F. 

**Print your final $\mathbf{F}$ matrix and $\mathbf{P}$ matrix.**

In [7]:
#set the value for d
d = -1

#create the p matrix
p = np.empty((len(seq1)+1,len(seq2)+1),dtype=str)

#loop through the matrix
for i in range(1, np.shape(f)[0]):
  for j in range(1,np.shape(f)[1]):
      up = f[i-1,j] + d
      left = f[i,j-1] + d
      diag = f[i-1,j-1] + s(seq1[i-1],seq2[j-1])
      f[i,j] = max(up,left,diag)
      if up == max(up,left,diag):
        p[i,j] = "up"
      if left == max(up,left,diag):
        p[i,j] = "left"
      if diag == max(up,left,diag):
        p[i,j] = "diag"

print("the P matrix is:")
print(p)
print("the F matrix is:")
print(f)

the P matrix is:
[['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' 'd' 'l' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l' 'l']
 ['' 'd' 'u' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'd' 'u' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'd' 'u' 'u' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l' 'l']
 ['' 'd' 'u' 'd' 'd' 'u' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'd' 'u' 'd' 'd' 'u' 'd' 'd' 'd' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'd' 'u' 'u' 'd' 'u' 'd' 'd' 'l' 'd' 'd' 'l' 'l']
 ['' 'd' 'u' 'd' 'd' 'u' 'd' 'd' 'u' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'u' 'u' 'u' 'u' 'u' 'd' 'l' 'u' 'd' 'l' 'l' 'd']]
the F matrix is:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13.]
 [ -1.   2.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10.]
 [ -2.   1.   4.   3.   2.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.]
 [ -3.   0.   3.   6.   5.   4.   3.   2.   1.   0.  -1.  -2.  -3.  -4.]
 [ -4.  -1.   2.   5.   8.   7.   6.   5.   4.  

### **e) Beginning at bottom corner, traceback the alignment and output the final alignment of the two sequences (10 points)**

At each step in the traceback we look at the entries of $\mathbf{P}_{i,j}$ matrix which records which cell (among $F_{i-1,j-1}, F_{i-1,j} and F_{i,j-1}$) $F_{i,j}$ was derived from. We take a 'step' back to the recorded previous entry that gave the max score, and add a pair of symbols/nucleotides to the alignment accordingly.

If $\mathbf{P}_{i,j}$ is diag, the step is to $F_{i-1,j-1}$ (diagonal arrow), $x_i$ and $y_j$ are added to the alignment. 

If $\mathbf{P}_{i,j}$ is up, the step is to $F_{i-1,j}$ (vertical arrow), and $x_i$ and '-' (gap character) are added.

If $\mathbf{P}_{i,j}$ is left, the step is to $F_{i,j-1}$ (horizontal arrow), and  '-' and $y_j$ are added.


If there is a tie for the max entry, choose any of the entries in that tie with equal probability. 

Example: here we combine the traceback and score matrix for visualization such that the numbers are the scores and the arrows represent the information in the traceback matrix.

Starting at the right corner, according to our recorded arrow, this comes from the left cell, thus T from y and a gap '-' are aligned. In the next step, again based on the arrow, the top cell is selected, thus the A from x is aligned to a gap '-'.

<center><img src="https://drive.google.com/uc?export=view&id=1mONEIOXBIBhTlYjevItK6dexR6V091cw" alt="exScore" width="450" height="350"><center>

The result alignment is:
```
G	-	A	T	T	A	-	C	A	-
G	C	A	T	-	-	G	C	-	T
```



In [8]:
def traceback(seq1,seq2,p):
  i = np.shape(p)[0]-1
  j = np.shape(p)[1]-1
  
  align_1 = ''
  align_2 = ''

  while (i > 0) and (j >0):
    if p[i,j] == 'l':
        align_1 = align_1 + '-'
        align_2 = align_2 + seq2[j-1]
        j -= 1
    if p[i,j] == 'u':
        align_1 = align_1 + seq1[i-1]
        align_2 = align_2 + '-'
        i -= 1
    if p[i,j] == 'd':
        align_1 = align_1 + seq1[i-1]
        align_2 = align_2 + seq2[j-1]
        j -= 1
        i -= 1
 
  return align_1[::-1],align_2[::-1]

In [9]:
seq1_allign, seq2_allign = traceback(seq1,seq2,p)
print("the allignment for sequence 1 is:", seq1_allign)
print("the allignment for sequence 2 is:", seq2_allign)

the allignment for sequence 1 is: ACAACA--A-CAG
the allignment for sequence 2 is: ACAACAGCAGCAG


### **f) Change the score function to adapt the mismatch penalties to account for more or less likely nucleotide substitutions. (5 points)**

Transitions within the purine class (A <--> G) or within the pyrimidine class (C <--> T) are more likely and thus have a **penalty of -1**. Transversions, across classes (e.g. A --> T) have a **penalty of -2** as they are less likely.

**Define a new s() function to implement these penalty changes.**

In [10]:
def s_new(b1,b2):
  s = -2
  if b1==b2:
    s = 2
  if (b1 == 'A' and b2 == 'G') or (b1 == 'G' and b2 == 'A'):
    s = -1
  if (b1 == 'C' and b2 == 'T') or (b1 == 'T' and b2 == 'C'):
    s = -1
  return s

### **g) Re-run the NW algorithm with new penalties, report your new final alignment with the penalty changes/new score function in f), and print the full (final) $\mathbf{F}$ matrix. (5 points)**

Use the adapted mismatch penalties. If there are multiple possible alignments, you just need to report one of them.

In [11]:
#initalize the new f matrix
f_new = np.zeros((len(seq1)+1,len(seq2)+1))

for i in range((np.shape(f_new)[0])-1):
  f_new[i+1,0] = f_new[i,0] -1

for i in range((np.shape(f_new)[1])-1):
  f_new[0,i+1] = f_new[0,i] -1

#create the p_new matrix
p_new = np.empty((len(seq1)+1,len(seq2)+1),dtype=str)

#loop through the matrix
for i in range(1, np.shape(f_new)[0]):
  for j in range(1,np.shape(f_new)[1]):
      up = f_new[i-1,j] + d
      left = f_new[i,j-1] + d
      diag = f_new[i-1,j-1] + s_new(seq1[i-1],seq2[j-1])
      f_new[i,j] = max(up,left,diag)
      if up == max(up,left,diag):
        p_new[i,j] = "up"
      if left == max(up,left,diag):
        p_new[i,j] = "left"
      if diag == max(up,left,diag):
        p_new[i,j] = "diag"

print("the P matrix is:")
print(p_new)
print("the F matrix is:")
print(f_new)

the P matrix is:
[['' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' 'd' 'l' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l' 'l']
 ['' 'd' 'u' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'd' 'u' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'd' 'u' 'u' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l' 'l']
 ['' 'd' 'u' 'd' 'd' 'u' 'd' 'l' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'd' 'u' 'd' 'd' 'u' 'd' 'd' 'l' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'd' 'u' 'u' 'd' 'u' 'u' 'd' 'l' 'l' 'd' 'l' 'l']
 ['' 'd' 'u' 'd' 'd' 'u' 'd' 'd' 'u' 'd' 'l' 'l' 'd' 'l']
 ['' 'u' 'u' 'u' 'u' 'u' 'u' 'd' 'u' 'u' 'd' 'l' 'l' 'd']]
the F matrix is:
[[  0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10. -11. -12. -13.]
 [ -1.   2.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.  -8.  -9. -10.]
 [ -2.   1.   4.   3.   2.   1.   0.  -1.  -2.  -3.  -4.  -5.  -6.  -7.]
 [ -3.   0.   3.   6.   5.   4.   3.   2.   1.   0.  -1.  -2.  -3.  -4.]
 [ -4.  -1.   2.   5.   8.   7.   6.   5.   4.  

In [12]:
#now lets perform our allignment
seq1_allign_new, seq2_allign_new = traceback(seq1,seq2,p_new)
print("the allignment for sequence 1 is:", seq1_allign_new)
print("the allignment for sequence 2 is:", seq2_allign_new)

the allignment for sequence 1 is: ACAACA--A-CAG
the allignment for sequence 2 is: ACAACAGCAGCAG


It looks like the new base pair switching rules did not change the allignment!